In [30]:
import seaborn as sns
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path

from functools import reduce

### Extraemos la informacion a dataframes para su uso

In [31]:
# -------------------------------------------------------------------------
# Define project paths
# If the notebook runs inside `notebooks/` (case-insensitive),
# move one level up to project root. Otherwise, use the current working directory.
# -------------------------------------------------------------------------
cwd_name = Path.cwd().name.lower()
PROJECT_ROOT = Path.cwd().parents[0] if cwd_name == "notebooks" else Path.cwd()

# Define subdirectories for data
DATA_RAW = PROJECT_ROOT / "data" / "raw"              # for raw example datasets
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"  # for cleaned / processed data

# Ensure directories exist
DATA_RAW.mkdir(parents=True, exist_ok=True)
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

# -------------------------------------------------------------------------
# Example file paths (CSV placeholders for testing/demo purposes)
# -------------------------------------------------------------------------
SYNTHETIC_CSV = DATA_RAW / "synthetic.csv"             # Example synthetic dataset
SP500_CSV     = DATA_RAW / "sp500_example.csv"         # Example: columns = Date, SP500_Price
BTC_CSV       = DATA_RAW / "bitcoin_example.csv"       # Example: columns = Date, BTC_Price
FX_CSV        = DATA_RAW / "fx_eur_usd_example.csv"    # Example: columns = Date, EURUSD



In [32]:
# -------------------------------------------------------------------------
# Load datasets from CSV files
# -------------------------------------------------------------------------

def safe_read_csv(path: Path, name: str) -> pd.DataFrame:
    """Try to read a CSV file; raise a clear error if not found."""
    if not path.exists():
        raise FileNotFoundError(f"Missing file for {name}: {path}")
    try:
        return pd.read_csv(path)
    except Exception as e:
        raise ValueError(f"Error reading {name} from {path}: {e}")

# Read datasets
df = safe_read_csv(SYNTHETIC_CSV, "synthetic dataset")
df_sp500 = safe_read_csv(SP500_CSV, "S&P 500 dataset")
df_btc = safe_read_csv(BTC_CSV, "Bitcoin dataset")
df_fx = safe_read_csv(FX_CSV, "FX EUR/USD dataset")



#### Limpieza de los dataframes

In [33]:
# -------------------------------------------------------------------------
# Helpers to standardize DataFrames
# -------------------------------------------------------------------------

def normalize_columns(pdf: pd.DataFrame) -> pd.DataFrame:
    """
    Return a copy with cleaned column names:
    - strip spaces
    - lowercase
    - replace spaces with underscores
    - remove special characters
    """
    out = pdf.copy()
    out.columns = (
        out.columns
        .str.strip()
        .str.lower()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^\w_]", "", regex=True)
    )
    return out

def coerce_datetime(series: pd.Series) -> pd.Series:
    """Convert a pandas Series to datetime with error coercion."""
    return pd.to_datetime(series, errors="coerce")

# -------------------------------------------------------------------------
# Normalize columns
# -------------------------------------------------------------------------
df_fx    = normalize_columns(df_fx)
df_btc   = normalize_columns(df_btc)
df_sp500 = normalize_columns(df_sp500)
df       = normalize_columns(df)

# -------------------------------------------------------------------------
# Subset and rename relevant columns (market data)
# -------------------------------------------------------------------------

# FX dataset: keep only date and EUR/USD
df_fx = df_fx[["date", "euro_to_usd"]].rename(columns={"euro_to_usd": "eurusd"})

# Bitcoin dataset
df_btc = df_btc[["open_time", "open"]].rename(columns={"open_time": "date", "open": "btc_price"})

# S&P 500 dataset
df_sp500 = df_sp500[["date", "open"]].rename(columns={"open": "sp500_price"})

# -------------------------------------------------------------------------
# Synthetic dataset: rename to English and filter
# -------------------------------------------------------------------------
df = df.rename(columns={"fecha": "date", "importe": "amount", "categoria": "category"})

# Validate required columns
required_cols = {"date", "amount", "category"}
missing = required_cols - set(df.columns)
if missing:
    raise KeyError(f"Synthetic dataset missing columns: {sorted(missing)}")

# Normalize category names and keep only 'ahorro'
df["category"] = df["category"].astype(str).str.strip().str.lower()
df = df[df["category"] == "ahorro"].copy()

# Parse date (many Spanish CSVs use dd/mm/yyyy format)
df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)

# Ensure amount is numeric; take absolute values if only deposits are expected
df["amount"] = pd.to_numeric(df["amount"], errors="coerce").abs()

# Debug checks
print("Rows after category filter:", len(df))
print("Date range:", df["date"].min(), "→", df["date"].max())
print("Null dates:", df["date"].isna().sum())

# -------------------------------------------------------------------------
# Ensure market dates are datetime
# -------------------------------------------------------------------------
for dataset in [df_sp500, df_btc, df_fx]:
    dataset["date"] = coerce_datetime(dataset["date"])

# -------------------------------------------------------------------------
# Ensure numeric types on price columns
# -------------------------------------------------------------------------
for col_df, cols in [
    (df_sp500, ["sp500_price"]),
    (df_btc,   ["btc_price"]),
    (df_fx,    ["eurusd"]),
]:
    for c in cols:
        col_df[c] = pd.to_numeric(col_df[c], errors="coerce")




Rows after category filter: 63
Date range: 2019-06-01 00:00:00 → 2024-04-17 00:00:00
Null dates: 0


C:\Users\diego\AppData\Local\Temp\ipykernel_9756\2307566289.py:64: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)


#### Llenamos los datos vacios con los superiores e inferiores 

In [34]:
# -------------------------------------------------------------------------
# Merge market datasets into a single frame aligned by 'date'
# - Uses left joins to keep all rows from the synthetic dataset (df)
# - Sorts by date and forward/backward fills missing market prices
# -------------------------------------------------------------------------


# In case any source has multiple rows per date, collapse to one row per date first
# (keep the last available value for that date).
df_sp500 = df_sp500.sort_values("date").groupby("date", as_index=False).agg({"sp500_price": "last"})
df_btc   = df_btc.sort_values("date").groupby("date", as_index=False).agg({"btc_price": "last"})
df_fx    = df_fx.sort_values("date").groupby("date", as_index=False).agg({"eurusd": "last"})


# Merge all on 'date' with left joins, keeping synthetic rows as the anchor
to_merge = [df, df_sp500, df_btc, df_fx]
prices_df = reduce(lambda left, right: pd.merge(left, right, on="date", how="left"), to_merge)

# Sort by date and reset index
prices_df = prices_df.sort_values("date").reset_index(drop=True)

# Forward-fill and back-fill typical market-price columns to bridge weekends/holidays
price_cols = [c for c in ["eurusd", "btc_price", "sp500_price"] if c in prices_df.columns]
if price_cols:
    prices_df[price_cols] = prices_df[price_cols].ffill().bfill()


#### Convertimos nuestros precios de bitcoin y S&P500 a euros con el cambio correspondient al dia

In [35]:
# -------------------------------------------------------------------------
# Convert market prices to EUR using the EUR/USD exchange rate
# -------------------------------------------------------------------------

prices_df["btc_price"]   = prices_df["btc_price"]   * prices_df["eurusd"]
prices_df["sp500_price"] = prices_df["sp500_price"] * prices_df["eurusd"]

# Drop the exchange rate column (no longer needed after conversion)
prices_df = prices_df.drop(columns="eurusd")

# Inspect resulting schema
prices_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63 entries, 0 to 62
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         63 non-null     datetime64[ns]
 1   amount       63 non-null     float64       
 2   category     63 non-null     object        
 3   sp500_price  63 non-null     float64       
 4   btc_price    63 non-null     float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 2.6+ KB


### Funcion con la que creamos la simulacion, por entrada

In [36]:
def simulate_investment_evolution(
    bitcoin_pct: float,
    fund_pct: float,
    data: pd.DataFrame,
    amount_col: str = "amount",
    date_col: str = "date",
    sp_col: str = "sp500_price",
    btc_col: str = "btc_price",
) -> pd.DataFrame:
    """
    Simulate the portfolio evolution given periodic contributions split into
    cash, S&P 500, and Bitcoin allocations.

    Parameters
    ----------
    bitcoin_pct : float
        Percentage allocated to Bitcoin (0–100).
    fund_pct : float
        Percentage allocated to S&P 500 (0–100).
    data : pd.DataFrame
        Input frame containing at least the contribution amount, date, and price columns.
    amount_col : str, default "amount"
        Column name for contribution amounts (positive numbers recommended).
    date_col : str, default "date"
        Column name for the date.
    sp_col : str, default "sp500_price"
        Column name for the S&P 500 price (in EUR in your pipeline).
    btc_col : str, default "btc_price"
        Column name for the Bitcoin price (in EUR in your pipeline).

    Returns
    -------
    pd.DataFrame
        DataFrame with [date_col, 'portfolio_value'] showing the cumulative value over time.
    """

    # --------- Basic validation ---------
    if any(col not in data.columns for col in [amount_col, date_col, sp_col, btc_col]):
        missing = [c for c in [amount_col, date_col, sp_col, btc_col] if c not in data.columns]
        raise KeyError(f"Missing required columns: {missing}")

    if bitcoin_pct < 0 or fund_pct < 0:
        raise ValueError("Allocations cannot be negative. I checked. Physics gets annoyed.")
    if bitcoin_pct + fund_pct > 100:
        raise ValueError("Allocations must sum to at most 100 (cash is the remainder).")

    # --------- Setup & allocations ---------
    cash_pct = (100 - (fund_pct + bitcoin_pct)) / 100.0
    btc_pct = bitcoin_pct / 100.0
    fund_pct = fund_pct / 100.0

    df = data.copy()

    # Ensure positive contributions; if you do refunds/withdrawals, remove .abs()
    df[amount_col] = df[amount_col].abs()

    # --------- Units purchased per contribution ---------
    df["cash_flow"] = df[amount_col] * cash_pct
    df["sp_units"] = (df[amount_col] * fund_pct) / df[sp_col]
    df["btc_units"] = (df[amount_col] * btc_pct) / df[btc_col]

    # --------- Accumulated positions ---------
    df["cash_cum"] = df["cash_flow"].cumsum()
    df["sp_units_cum"] = df["sp_units"].cumsum()
    df["btc_units_cum"] = df["btc_units"].cumsum()

    # --------- Mark-to-market portfolio value ---------
    df["portfolio_value"] = (
        df["cash_cum"]
        + df["sp_units_cum"] * df[sp_col]
        + df["btc_units_cum"] * df[btc_col]
    )

    # Output only what you need
    return df[[date_col, "portfolio_value"]].sort_values(by=date_col).reset_index(drop=True)


### Combinamos las distintas distribuciones con la simulacion de precios


In [37]:
def simulate_investment_evolution_for_all_options(
    options_df: pd.DataFrame,
    data_df: pd.DataFrame,
    btc_col: str = "BTC",
    sp_col: str = "SP",
) -> pd.DataFrame:
    """
    Run investment evolution simulations for all allocation combinations
    provided in `options_df`.

    Parameters
    ----------
    options_df : pd.DataFrame
        DataFrame with allocation options. Must contain at least two columns:
        - btc_col : percentage allocation to Bitcoin (0–100)
        - sp_col  : percentage allocation to S&P 500 (0–100)
    data_df : pd.DataFrame
        Market + synthetic contributions dataset (already standardized).
    btc_col : str, default "BTC"
        Name of the Bitcoin allocation column in options_df.
    sp_col : str, default "SP"
        Name of the S&P 500 allocation column in options_df.

    Returns
    -------
    pd.DataFrame
        Concatenated simulation results for all allocation options.
        Includes 'BTC' and 'SP' as columns identifying each simulation.
    """

    results = {}

    for _, row in options_df.iterrows():
        btc_pct = row[btc_col]
        sp_pct = row[sp_col]

        evolution = simulate_investment_evolution(
            bitcoin_pct=btc_pct,
            fund_pct=sp_pct,
            data=data_df
        )
        results[(btc_pct, sp_pct)] = evolution

    # Concatenate with MultiIndex keys and reset
    results = pd.concat(results, names=[btc_col, sp_col]).reset_index(level=[0, 1])

    return results


### Creamos un dataframe con todas las opciones de inversion

In [38]:
# -------------------------------------------------------------------------
# Generate allocation combinations for BTC and S&P 500
# - Step size can be adjusted with `step`
# - Keeps only combinations where BTC + SP <= 100
# -------------------------------------------------------------------------

step = 10
btc_values = range(0, 101, step)
sp_values = range(0, 101, step)

# Build all combinations and filter by total <= 100
allocations = [
    (btc, sp)
    for btc in btc_values
    for sp in sp_values
    if btc + sp <= 100
]

# Create DataFrame of allocation options
investment_options_df = pd.DataFrame(allocations, columns=["BTC", "SP"])


### Ejecutamos la simulacion

In [39]:
# -------------------------------------------------------------------------
# Run simulations for all allocation options and store results
# -------------------------------------------------------------------------

results_df = simulate_investment_evolution_for_all_options(
    options_df=investment_options_df,
    data_df=prices_df
)

results_df




,BTC,SP,date,portfolio_value
0,0,0,2019-06-01,250.000000
1,0,0,2019-07-01,500.000000
2,0,0,2019-07-29,510.000000
3,0,0,2019-07-31,530.000000
4,0,0,2019-08-01,780.000000
...,...,...,...,...
58,100,0,2023-08-01,42090.633276
59,100,0,2023-12-05,60768.428058
60,100,0,2024-03-01,89034.465059
61,100,0,2024-03-07,97463.574717


### Obtenemos todas las opciones para la ultima fecha del dataframe

In [40]:
# -------------------------------------------------------------------------
# Get portfolio values at the last available date
# -------------------------------------------------------------------------

# Find the most recent date in the results
last_date = results_df["date"].max()

# Filter rows corresponding to the last date
df_last_date = results_df[results_df["date"] == last_date]


### Grafico temporal de algunas de las opciones de inversion

In [41]:
# -------------------------------------------------------------------------
# Visualization: compare selected portfolio allocation strategies
# -------------------------------------------------------------------------

# Ensure date column is datetime
results_df["date"] = pd.to_datetime(results_df["date"])

# Create a label for each allocation combination
results_df["allocation_label"] = results_df.apply(
    lambda row: f"BTC {row['BTC']}% - SP {row['SP']}%", axis=1
)

# Filter only the allocations we want to highlight
selected_allocations = [
    "BTC 0% - SP 0%",
    "BTC 50% - SP 50%",
    "BTC 100% - SP 0%",
    "BTC 0% - SP 100%",
]
filtered_results = results_df[results_df["allocation_label"].isin(selected_allocations)]

# Line chart with markers for each allocation
fig = px.line(
    filtered_results,
    x="date",
    y="portfolio_value",
    color="allocation_label",
    markers=True,
    title="Investment Evolution Over Time",
    labels={"portfolio_value": "Portfolio Value (EUR)", "date": "Date"},
    template="ggplot2"
)

# Adjust legend to appear outside the chart (to the right)
fig.update_layout(
    legend=dict(
        title="Investment Allocation",
        x=1.05,  # horizontal position
        y=1,     # vertical position (top)
        xanchor="left"
    )
)

fig.show()


### Creamos la matriz, comparando las opciones de inversion a la ultima fecha del dataframe

In [42]:
# -------------------------------------------------------------------------
# Build a pivot table (matrix) of final portfolio values
# Rows = SP allocation, Columns = BTC allocation
# -------------------------------------------------------------------------

matrix = pd.crosstab(
    index=df_last_date["SP"],
    columns=df_last_date["BTC"],
    values=df_last_date["portfolio_value"],
    aggfunc="sum"
)


In [43]:
matrix

BTC,0,10,20,30,40,50,60,70,80,90,100
SP,,,,,,,,,,,
0,29670.190000,35971.149601,42272.109201,48573.068802,54874.028403,61174.988003,67475.947604,73776.907205,80077.866805,86378.826406,92679.786007
10,30406.496597,36707.456197,43008.415798,49309.375399,55610.334999,61911.294600,68212.254201,74513.213801,80814.173402,87115.133003,NaN
20,31142.803193,37443.762794,43744.722395,50045.681995,56346.641596,62647.601197,68948.560797,75249.520398,81550.479999,NaN,NaN
30,31879.109790,38180.069391,44481.028991,50781.988592,57082.948193,63383.907793,69684.867394,75985.826995,NaN,NaN,NaN
40,32615.416387,38916.375987,45217.335588,51518.295189,57819.254789,64120.214390,70421.173991,NaN,NaN,NaN,NaN
50,33351.722983,39652.682584,45953.642185,52254.601785,58555.561386,64856.520987,NaN,NaN,NaN,NaN,NaN
60,34088.029580,40388.989181,46689.948781,52990.908382,59291.867983,NaN,NaN,NaN,NaN,NaN,NaN
70,34824.336177,41125.295777,47426.255378,53727.214979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,35560.642773,41861.602374,48162.561975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# -------------------------------------------------------------------------
# Style the allocation matrix for better readability
# -------------------------------------------------------------------------

styled_matrix = (
    matrix.style
    .background_gradient(cmap="viridis", axis=None)  # apply gradient to entire DataFrame
    .format(na_rep="")  # hide NaN values
)

# Display styled matrix
styled_matrix



BTC,0,10,20,30,40,50,60,70,80,90,100
SP,,,,,,,,,,,
0,29670.190000,35971.149601,42272.109201,48573.068802,54874.028403,61174.988003,67475.947604,73776.907205,80077.866805,86378.826406,92679.786007
10,30406.496597,36707.456197,43008.415798,49309.375399,55610.334999,61911.294600,68212.254201,74513.213801,80814.173402,87115.133003,
20,31142.803193,37443.762794,43744.722395,50045.681995,56346.641596,62647.601197,68948.560797,75249.520398,81550.479999,,
30,31879.109790,38180.069391,44481.028991,50781.988592,57082.948193,63383.907793,69684.867394,75985.826995,,,
40,32615.416387,38916.375987,45217.335588,51518.295189,57819.254789,64120.214390,70421.173991,,,,
50,33351.722983,39652.682584,45953.642185,52254.601785,58555.561386,64856.520987,,,,,
60,34088.029580,40388.989181,46689.948781,52990.908382,59291.867983,,,,,,
70,34824.336177,41125.295777,47426.255378,53727.214979,,,,,,,
80,35560.642773,41861.602374,48162.561975,,,,,,,,


### HeatMap con la matriz para visualizar la evolucion de la inversion segun las opciones

In [45]:
# -------------------------------------------------------------------------
# Heatmap of final portfolio values by BTC vs S&P500 allocation
# -------------------------------------------------------------------------

fig = px.imshow(
    matrix,
    text_auto=".2f",  # show values with 2 decimals
    color_continuous_scale="Viridis"
)

fig.update_layout(
    title="Investment Allocation: S&P500 vs Bitcoin",
    xaxis_title="Bitcoin Allocation (%)",
    yaxis_title="S&P500 Allocation (%)",
    template="ggplot2",
    width=800,
    height=600,
    margin=dict(l=10, r=10, t=50, b=10)  # adjust margins
)

fig.show()
